>### The code below takes multiple hd5 files from the Munn survey as input and outputs an array with the mean offsets of DDO51.

In [2]:
#Counting number of files in a directory and a list of all hd5 file names.  This will be used to create a blank array for mean offsets.

# Import modules
from astropy.io import fits
from astropy.table import Table
import numpy as np
import os, os.path

path, dirsyear, files = next(os.walk(r"C:\Users\shebe\OneDrive\School\Graduate\Research\Nidever\Data\DDO51"))

nights = []

for i in dirsyear:
    path, dirsnight, files = next(os.walk(r"C:\Users\shebe\OneDrive\School\Graduate\Research\Nidever\Data\DDO51\{0}".format(i)))
    for j in dirsnight:
        path, dirs, filesnight = next(os.walk(r"C:\Users\shebe\OneDrive\School\Graduate\Research\Nidever\Data\DDO51\{0}\{1}\pcat".format(i,j)))
        nights += filesnight

#The variable dirs contains the names of all the folders containing hd5 files.  
#This can be referenced during construction of the array

files = np.array(nights)
mean = np.zeros(len(nights))
col1 = fits.Column(name='file', format='A15', array=files)
col2 = fits.Column(name='mean', format='D05', unit='DN', array=mean)
coldefs = fits.ColDefs([col1, col2])
moff = fits.BinTableHDU.from_columns(coldefs)
len(moff.data['file'])

#For loop to append mean offsets to array for each file
for i in nights:
    #Loading data from night one
    z = Table.read(r'C:\Users\shebe\OneDrive\School\Graduate\Research\Nidever\Data\DDO51\20{0}\{1}\pcat\{2}'.format(i[1:3],i[0:7],i))

    #loading dwarf locus data
    dat = Table.read(r'C:\Users\shebe\OneDrive\School\Graduate\Research\Nidever\Data\gaiad51_dwarflocus.fits')

    #Removing nans from Gaia BP and RP matched photometry
    ind = np.isnan(z['phot_bp_mean_mag'])
    tb = z['phot_bp_mean_mag']
    tr = z['phot_rp_mean_mag']
    dd = z['ddo51']
    ind = np.where((~np.isnan(tb))&(~np.isnan(tr))&(~np.isnan(dd)))

    #Remove any row with a nan in BP or RP
    tb = tb[ind]
    tr = tr[ind]
    dd = dd[ind]
    
    #Zero-point data
    #Calculate differences between photometric bands
    bprp = tb-tr
    bpd = tb-dd
    dt = np.dtype([('bprp',float),('bpd',float)])
    zerop = np.zeros(len(dd), dtype=dt)
    zerop['bprp'] = bprp
    zerop['bpd'] = bpd
    
    #Focus on values near the dwarf locus
    ind = np.where((zerop['bprp']<3)&(zerop['bpd']<3))
    zerop = zerop[ind]
    hist, xedges, yedges = np.histogram2d(zerop['bprp'],zerop['bpd'], [50,50])
    ind = np.where(hist == hist.max())
    
    #Find the location of highest density in the locus
    x0 = xedges[ind[0]][0]
    y0 = yedges[ind[1]][0]
    good = np.where((zerop['bpd']<(y0+0.15))&(zerop['bpd']>(y0-0.68))&(zerop['bprp']<(x0+2.17))&(zerop['bprp']>(x0-0.574)))
    zerop = zerop[good]
    
    #Calculate mean offset for field and append to fits table moff
    photint = np.interp(zerop['bprp'],dat['BPRP'],dat['BPD'])
    diff = zerop['bpd']  - photint
    m = np.mean(diff)
    loc = np.where(moff.data['file']==i)
    moff.data['mean'][loc] = m

moff.data['mean']

array([0.11686023, 0.05771012, 0.06551465, ..., 0.04954515, 0.03667743,
       0.09239255])

In [9]:
#Viewing array
t = moff.data

In [8]:
#Maximum and minimum mean offsets
print('Minimum mean offset: {0}'.format(min(t['mean'])))
print('Maximum mean offset: {0}'.format(max(t['mean'])))
print('Median: {0}'.format(np.median(t['mean'])))
print('Mean of all means: {0}'.format(np.mean(t['mean'])))

Minimum mean offset: -100.50419902311815
Maximum mean offset: 1.0671678294739355
Median: 0.0783413076726828
Mean of all means: 0.03244718999507937


In [13]:
t = t[np.where(t['mean']<-2)]

In [14]:
t

FITS_rec([('g12d227.013.h5',  -97.92425986),
          ('g13d189.018.h5',  -97.96929635),
          ('g13d245.013.h5',  -98.36835987),
          ('g14d154.021.h5', -100.50419902),
          ('g17d141.077.h5',  -97.63134225)],
         dtype=(numpy.record, [('file', 'S15'), ('mean', '<f8')]))